# Jsonifying Teamates csv's for Dashboard use

In [71]:
# Imports
import pandas as pd
from pathlib import Path
import json
import pprint as pp
import os
import us
import pymongo

# Sean's Gunsales data
Since csv is cleaned up, Jsonify it

In [72]:
# Store filepath in a variable
file = Path("../Sean/Resources/us_gun_sales_data_COMPLETE.csv")
gunsales_df = pd.read_csv(file)
gunsales_df.head()

,Province_State,2023 Total Estimated Sales,Guns Per Capita
0,Alabama,487077,9.53
1,Alaska,80341,10.95
2,Arizona,407288,5.48
3,Arkansas,200279,6.53
4,California,1043421,2.68


In [73]:
# Initialize an empty list to store dictionaries
gunsales_list = []

# Iterate over the DataFrame rows
for index, row in gunsales_df.iterrows():
    # Create a dictionary with the hospital name and other information
    gunsales_dict = {
        'State': row['Province_State'],
        'Gun Sales Info': row.drop(labels=['Province_State']).to_dict()
    }
    # Append the dictionary to the list
    gunsales_list.append(gunsales_dict)

# Convert the list of dictionaries to a JSON-formatted string
gunsales_json = json.dumps(gunsales_list, indent=4)

# Print the JSON-formatted string
print(gunsales_json)

[
    {
        "State": "Alabama",
        "Gun Sales Info": {
            "2023 Total Estimated Sales": 487077,
            "Guns Per Capita": 9.53
        }
    },
    {
        "State": "Alaska",
        "Gun Sales Info": {
            "2023 Total Estimated Sales": 80341,
            "Guns Per Capita": 10.95
        }
    },
    {
        "State": "Arizona",
        "Gun Sales Info": {
            "2023 Total Estimated Sales": 407288,
            "Guns Per Capita": 5.48
        }
    },
    {
        "State": "Arkansas",
        "Gun Sales Info": {
            "2023 Total Estimated Sales": 200279,
            "Guns Per Capita": 6.53
        }
    },
    {
        "State": "California",
        "Gun Sales Info": {
            "2023 Total Estimated Sales": 1043421,
            "Guns Per Capita": 2.68
        }
    },
    {
        "State": "Colorado",
        "Gun Sales Info": {
            "2023 Total Estimated Sales": 443800,
            "Guns Per Capita": 7.55
        }
    },
   

In [74]:
# Path to the resources folder
file_path = os.path.join('Resources', 'cleaned_gunsales.json')

# Write the list of dictionaries to a JSON file in the resources folder
with open(file_path, 'w') as json_file:
    json.dump(gunsales_list, json_file, indent=4)

print(f"JSON data has been written to {file_path}")

JSON data has been written to Resources/cleaned_gunsales.json


# Levi's Military Bases Data
Cleaning CSV

In [75]:
# Store filepath in a variable
file = Path("../Levi/military-bases.csv")
military_df = pd.read_csv(file, delimiter=';')

# Select desired columns only
military_selected_df = military_df[['Geo Point','COMPONENT','Site Name','State Terr','COUNTRY','Oper Stat']]
military_selected_df.head()

,Geo Point,COMPONENT,Site Name,State Terr,COUNTRY,Oper Stat
0,"31.2309993833, -85.6506347178",Army Active,Allen Stagefield AL,Alabama,United States,Active
1,"31.8157331822, -85.6497984957",Army Active,Louisville Stagefield AL,Alabama,United States,Active
2,"33.1594636742, -106.425696182",Army Active,White Sands Missile Range NM,New Mexico,United States,Active
3,"37.0130203962, -76.3043760544",Army Active,Fort Monroe,Virginia,United States,Inactive
4,"21.3866284869, -157.905641308",MC Active,MCB Camp Smith,Hawaii,United States,Active


In [76]:
# Filter for bases that are active only, and are in the US
military_active_df = military_selected_df[
    (military_selected_df['Oper Stat'] == 'Active') &
    (military_selected_df['COUNTRY'] == 'United States')
].copy()  

# Remove "Active" part after branch name in COMPONENT column
military_active_df['COMPONENT'] = military_active_df['COMPONENT'].str.replace('Active', '', regex=False).str.strip()

# Refine the branch names for readability
military_active_df['COMPONENT'] = military_active_df['COMPONENT'].str.replace('MC', 'Marine Corps', regex=False).str.strip()
military_active_df['COMPONENT'] = military_active_df['COMPONENT'].str.replace('AF', 'Air Force', regex=False).str.strip()
military_active_df['COMPONENT'] = military_active_df['COMPONENT'].str.replace('WHS', 'Washington Headquarters Services', regex=False).str.strip()

# Split the 'Geo Point' column into two columns: 'Latitude' and 'Longitude'
geo_points = military_active_df['Geo Point'].str.split(', ', expand=True)
military_active_df['Latitude'] = pd.to_numeric(geo_points[0])
military_active_df['Longitude'] = pd.to_numeric(geo_points[1])

# Drop the original 'Geo Point' column if no longer needed
military_filtered_df = military_active_df.drop(columns={
    'Geo Point', 'Oper Stat'})

# Rename columns
military_filtered_df = military_filtered_df.rename(columns={
    'COMPONENT': 'Military Branch',
    'State Terr': 'State',
    'COUNTRY': 'Country',
    'Oper Stat': 'Status'
})

# Reorder columns
new_order = ['Site Name', 'Military Branch', 'State','Country','Latitude','Longitude']

military_cleaned_df = military_filtered_df[new_order]

military_cleaned_df.head()

,Site Name,Military Branch,State,Country,Latitude,Longitude
0,Allen Stagefield AL,Army,Alabama,United States,31.230999,-85.650635
1,Louisville Stagefield AL,Army,Alabama,United States,31.815733,-85.649798
2,White Sands Missile Range NM,Army,New Mexico,United States,33.159464,-106.425696
4,MCB Camp Smith,Marine Corps,Hawaii,United States,21.386628,-157.905641
5,MC Trng Area Bellows,Marine Corps,Hawaii,United States,21.362146,-157.718266


Jsonify Data

In [77]:
# Initialize an empty list to store dictionaries
bases_list = []

# Iterate over the DataFrame rows
for index, row in military_filtered_df.iterrows():
    # Create a dictionary with the hospital name and other information
    military_bases_dict = {
        'State': row['State'],
        'Military Bases Info': row.drop(labels=['State']).to_dict()
    }
    # Append the dictionary to the list
    bases_list.append(military_bases_dict)

# Convert the list of dictionaries to a JSON-formatted string
military_bases_json = json.dumps(military_bases_dict, indent=4)

# Print the JSON-formatted string
print(military_bases_json)

{
    "State": "Nevada",
    "Military Bases Info": {
        "Military Branch": "Air Force",
        "Site Name": "Nellis Air Force Range",
        "Country": "United States",
        "Latitude": 37.3063972082,
        "Longitude": -116.198549994
    }
}


In [78]:
# Path to the resources folder
file_path = os.path.join('Resources', 'cleaned_military_bases.json')

# Write the list of dictionaries to a JSON file in the resources folder
with open(file_path, 'w') as json_file:
    json.dump(bases_list, json_file, indent=4)

print(f"JSON data has been written to {file_path}")

JSON data has been written to Resources/cleaned_military_bases.json
